In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# 因子分析用ライブラリ (未インストールの場合は !pip install factor_analyzer)
from factor_analyzer import FactorAnalyzer
import japanize_matplotlib

# 設定
plt.rcParams['font.size'] = 12
INPUT_FILE = "correlation_matrix.csv"

# カラム名の変換マッピング（元の長い質問文を、分析用の短い名称に変換）
rename_dict = {
    "年齢を教えてください。": "年齢",
    "年齢": "年齢",
    
    # 味覚項目
    "甘味をどの程度感じましたか？（１／５）": "甘味",
    "甘味": "甘味",
    "うま味をどの程度感じましたか？（２／５）": "うま味",
    "うま味": "うま味",
    "苦味をどの程度感じましたか？（３／５）": "苦味",
    "苦味": "苦味",
    "酸味をどの程度感じましたか？（４／５）": "酸味",
    "酸味": "酸味",
    "塩味をどの程度感じましたか？（５／５）": "塩味",
    "塩味": "塩味",
    
    # 視覚項目
    "華やかさをどの程度感じましたか？（１／５）": "華やかさ",
    "華やかさ": "華やかさ",
    "開放感をどの程度感じましたか？（２／５）": "開放感",
    "開放感": "開放感",
    "爽やかさをどの程度感じましたか？（３／５）": "爽やかさ",
    "爽やかさ": "爽やかさ",
    "賑い感をどの程度感じましたか？（４／５）": "賑わい感",
    "賑わい感をどの程度感じましたか？（４／５）": "賑わい感",
    "賑わい感": "賑わい感",
    "高揚感をどの程度感じましたか？（５／５）": "高揚感",
    "高揚感": "高揚感",
    
    # 目的変数
    "クラフトビール「白蒲田」をまた飲んでみたいと思いますか？": "リピート意向",
    "リピート意向": "リピート意向"
}

# 分析から除外する変数
# 因子分析は「観測変数の背後にある構成概念」を探るため、属性（年齢）や目的変数（リピート意向）は通常除外する
drop_columns = ["年齢", "リピート意向"]

# データの読み込み
try:
    correlation_matrix = pd.read_csv(INPUT_FILE, index_col=0)
    print(f"データ読み込み成功: {INPUT_FILE}")
except FileNotFoundError:
    print(f"エラー: ファイル '{INPUT_FILE}' が見つかりません。")
    # ダミーデータ（エラー回避用）
    cols = list(set(rename_dict.values()))
    correlation_matrix = pd.DataFrame(np.eye(len(cols)), index=cols, columns=cols)

# 変数名のリネーム
correlation_matrix.rename(index=rename_dict, columns=rename_dict, inplace=True)

# 不要な変数の除外
drop_targets = [col for col in drop_columns if col in correlation_matrix.columns]
df_analysis = correlation_matrix.drop(index=drop_targets, columns=drop_targets)

print("分析対象の相関行列:")
display(df_analysis.round(2))

# 因子数の決定（スクリープロット）
# ポリコリック相関行列を使用するため is_corr_matrix=True を指定
fa = FactorAnalyzer(rotation=None, is_corr_matrix=True)
fa.fit(df_analysis)

# 固有値の計算
ev, _ = fa.get_eigenvalues()

# スクリープロットの描画
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(ev)+1), ev, marker="o", linestyle="-", color="b")
plt.xlabel("因子の数")
plt.ylabel("固有値")
plt.title("Scree Plot")
# Kaiser基準（固有値=1）のライン
plt.axhline(y=1, color='r', linestyle='--', label="Eigenvalue = 1")
plt.legend()
plt.grid(True)
plt.show()

# 推奨因子数の表示
num_factors_kaiser = sum(ev > 1)
print(f"固有値1以上の因子数（Kaiser基準）: {num_factors_kaiser}")

# 因子分析の実行（複数パターンを確認）
# 例として 2因子 と 3因子 での結果を確認する（スクリープロットの結果を見て調整可）
# 回転法は解釈のしやすさから 'promax' (斜交回転) または 'quartimax/varimax' (直交回転) を選択
target_n_factors = [2, 3] 
rotation_method = "promax" 

for n in target_n_factors:
    print(f"\n{'='*40}")
    print(f" {n} 因子での分析結果 ({rotation_method}回転)")
    print(f"{'='*40}")

    # 因子分析の実行
    fa = FactorAnalyzer(n_factors=n, rotation=rotation_method, is_corr_matrix=True)
    fa.fit(df_analysis)

    # 因子負荷行列の取得
    factor_loadings = pd.DataFrame(
        fa.loadings_, 
        index=df_analysis.columns,
        columns=[f"Factor{i+1}" for i in range(n)]
    )

    # ヒートマップで可視化
    plt.figure(figsize=(10, 6))
    sns.heatmap(
        factor_loadings, 
        annot=True, 
        cmap="coolwarm", 
        fmt=".2f", 
        linewidths=0.5,
        center=0,
        vmax=1.0,
        vmin=-1.0
    )
    plt.title(f"因子負荷量 ({rotation_method}) - {n} Factors")
    plt.show()

    # 共通性と独自性の算出
    communalities = fa.get_communalities()
    uniquenesses = fa.get_uniquenesses()

    communalities_df = pd.DataFrame({
        "共通性": communalities,
        "独自性": uniquenesses
    }, index=df_analysis.columns)

    print("\n【共通性と独自性】")
    # 共通性が低い変数は、抽出された因子では説明しきれていないことを示す
    display(communalities_df.round(3))

    # 寄与率の算出
    variance_info = fa.get_factor_variance()
    variance_df = pd.DataFrame({
        "固有値": variance_info[0],
        "寄与率": variance_info[1],
        "累積寄与率": variance_info[2]
    }, index=[f"Factor{i+1}" for i in range(n)])

    print("\n【因子の寄与率】")
    display(variance_df.round(3))